# CTO Challenge 2015

ピザを買うときに使うクーポンの種類と枚数の最適解を求める

- [LEVEL 1](https://gist.github.com/makoga/4b993248de03032036a4)
- [LEVEL 2](https://gist.github.com/makoga/7a195de59388df32dc1d)
- [LEVEL 3](https://gist.github.com/makoga/3253f69c544b540eab00)

## LEVEL 1

整数計画問題として解けば良い

### 定式化

$Variables$
- $m$ クーポンの種類 $i = 1,....m$
- $\mathit{CouponPrice}_i$ クーポンの価格
- $\mathit{CouponPosessions}_i$ クーポンの手持ち数
- $\mathit{CouponUse}_i \geq 0$ クーポンの利用枚数
- $\mathit{TotalPrice}$ 金額

$Maximize$  
- $\displaystyle {\sum_i \{\mathit{CouponPrice}_i \times \mathit{CouponUse}_i}\} - \sum_i\mathit{CouponUse}_i$

$Subject\ to$

- $\sum_i \{\mathit{CouponPrice}_i \times \mathit{CouponUse_i}\} \leq \mathit{TotalPrice}$
- $\mathit{CouponUse}_i \leq \mathit{CouponPosessions_i}$
- $\mathit{TotalPrice} - 1000 \geq \sum_j \mathit{CouponUse}$

In [1]:
import pulp

In [137]:
def solve_level_1(total_price, coupon_posession):
    """
    Parameters
    ==========
    total_price : int
        注文金額
    coupon_posession : array(int)
        クーポン毎の所持数
        
    Returns
    =======
    coupon_use : array(int)
        クーポン毎の使用数
    """
    # クーポン金額
    coupon_price = [500, 200, 100]
    m = len(coupon_price)

    problem = pulp.LpProblem(sense=pulp.LpMaximize)
    # 変数
    coupon_use = [pulp.LpVariable('CouponUse{0}'.format(i), cat=pulp.LpInteger, lowBound=0) for i in range(m)]
    # 問題
    problem += pulp.lpDot(coupon_use, coupon_price) - pulp.lpSum(coupon_use)
    # 制約条件
    problem += pulp.lpDot(coupon_use, coupon_price) <= total_price
    problem += total_price - 1000 >= pulp.lpSum(coupon_use)
    for i in range(m):
        problem += coupon_posession[i] - coupon_use[i] >= 0
    
    status = problem.solve()
    print(pulp.LpStatus[status])
    #print(problem)
    return [coupon_use[i].value() for i in range(m)]

In [138]:
solve_level_1(1210, [2, 1, 3])

Optimal


[2.0, 1.0, 0.0]

In [92]:
# テストケース
assert solve_level_1(1000, [2, 1, 3]) == [0, 0, 0]
assert solve_level_1(1210, [0, 0, 0])  == [0, 0, 0]
assert solve_level_1(1210, [2, 1, 3])  == [2, 1, 0]
assert solve_level_1(1530, [2, 1, 3])  == [2, 1, 3]
# 枚数が一番小さい組み合わせを選択する事の確認
assert solve_level_1(1500, [3, 15, 15])  == [3, 0, 0]

Optimal
Optimal
Optimal
Optimal
Optimal


## LEVEL 2

変数の追加 or 変更

- $\mathit{CouponPrice_i} = [500, 200, 100, 300]$
- $n$ メニューの種類数 $j = 1,....n$
- $\mathit{MenuPrice_j} = [1000, 1500, 1300, 1800, 400, 500, 600]$ メニューの価格
- $\mathit{IsPizza_j} = [1, 1, 1, 1, 0, 0, 0]$ ピザフラグ
- $\mathit{Order_j} \geq 0$ 各メニューの注文数
- $\mathit{TotalPrice} = \sum_j \{\mathit{MenuPrice_j} \times \mathit{Order_j}\}$ 金額


追加制約
- $\mathit{CouponUse_1} \leq 2$
- $\mathit{CouponUse_2} \leq 2$
- $\mathit{CouponUse_3} \leq 3$
- $\mathit{CouponUse_4} \leq 1$
- $\mathit{CouponUse_4} \leq Order \cdot IsPizza$

In [73]:
def solve_level_2(order, coupon_posession):
    """
    Parameters
    ==========
    order : array(int)
        注文 (メニュー毎の注文数を表現するベクトル)
    coupon_posession : array(int)
        クーポン所持数
        
    Returns
    =======
    coupon_use : array(int)
        クーポン毎の使用数
    """
    # クーポン金額
    coupon_price = [500, 200, 100, 300]
    m = len(coupon_price)
    # メニュー価格
    menu_price = [1000, 1500, 1300, 1800, 400, 500, 600]
    # Pizzaフラグ
    is_pizza = [1, 1, 1, 1, 0, 0, 0]
    # 支払い金額
    total_price = np.dot(menu_price, order)

    problem = pulp.LpProblem(sense=pulp.LpMaximize)
    # 変数
    coupon_use = [pulp.LpVariable('CouponUse{0}'.format(i), cat=pulp.LpInteger, lowBound=0) for i in range(m)]
    # 問題
    problem += pulp.lpDot(coupon_use, coupon_price) - pulp.lpSum(coupon_use)
    # 制約条件
    problem += pulp.lpDot(coupon_use, coupon_price) <= total_price
    problem += pulp.lpSum(coupon_use) <= (total_price - 1000) 
    for i in range(m):
        problem += coupon_posession[i] - coupon_use[i] >= 0
    problem += coupon_use[0] <= 2
    problem += coupon_use[1] <= 2
    problem += coupon_use[2] <= 3
    problem += coupon_use[3] <= 1
    problem += coupon_use[3] <= np.dot(order, is_pizza)

    status = problem.solve()
    print(pulp.LpStatus[status])
    #print(problem)
    return [coupon_use[i].value() for i in range(m)]

In [75]:
solve_level_2([0, 0, 0, 0, 2, 1, 0], [2, 1, 1, 1])

Optimal


[2.0, 1.0, 1.0, 0.0]

In [76]:
# テストケース
# ジェノベーゼMx1
assert solve_level_2([1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1]) == [0, 0, 0, 0]
# マルゲリータMx1
assert solve_level_2([0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0]) == [0, 0, 0, 0]
# ポテトフライx1, グリーンサラダx1
assert solve_level_2([0, 0, 0, 0, 2, 1, 0], [2, 1, 1, 1]) == [2, 1, 1, 0]
# マルゲリータMx1
assert solve_level_2([0, 0, 1, 0, 0, 0, 0], [2, 1, 1, 1]) == [2, 0, 0, 1]
# ジェノベーゼMx1, マルゲリータMx1
assert solve_level_2([1, 0, 1, 0, 0, 0, 0], [3, 3, 4, 2]) == [2, 2, 3, 1]

Optimal
Optimal
Optimal
Optimal
Optimal


## LEVEL 3

- ピザ2セット: ポテトフライが1増える
- ピザL2セット: 任意のサイドオーダーを1増やせる

欲しい物が満たされる時の支払い金額を最小化する

- $Requirements_j$ 必要なメニュー
- $Order_j \geq 0$ 注文するメニュー
- $OrderPrice = \sum_j Order_jMenuPrice_j$ 注文金額
- $Payout = \sum_j Order_jMenuPrice_j - \sum CouponUse_iCouponPrice_i$ 請求金額
- $IsLPizza = [0, 1, 0, 1, 0, 0, 0]$
- $IsSide = [0, 0, 0, 0, 1, 1, 1]$
- $Poteto = [0, 0, 0, 0, 1, 0, 0]$
- $Piza2SetUse \geq 0$
- $Piza2SetItem = [0, 0, 0, 0, Piza2SetUse, 0, 0]$
- $PizaL2SetUse \geq 0$
- $PizaL2SetItem = [0, 0, 0, 0, x, y, z]$

$Minimize$

$Payout + \sum_j CouponUse$

追加制約

- $\displaystyle Piza2SetUse \leq \frac{Order_j \cdot IsPizza_j}{2}$
- $\displaystyle PizaL2SetUse \leq \frac{Order_j \cdot IsLPizza_j}{2}$
- $\displaystyle PizaL2SetUse = \sum_j PizaL2SetItem$ 
- $Requirements_j = Order_j + Piza2SetItem + PizaL2SetItem$

In [127]:
def solve_level_3(requirements, coupon_posession):
    """
    Parameters
    ==========
    requirements : array(int)
        欲しいメニュー (メニュー毎の注文数を表現するベクトル)
    coupon_posession : array(int)
        クーポン所持数
        
    Returns
    =======
    coupon_use : array(int)
        クーポン毎の使用数
    """
    #######################################################
    # 定数
    #######################################################
    # クーポン金額
    coupon_price = [500, 200, 100, 300]
    m = len(coupon_price)
    # メニュー価格
    menu_price = [1000, 1500, 1300, 1800, 400, 500, 600]
    n = len(menu_price)
    # フラグ
    is_pizza = [1, 1, 1, 1, 0, 0, 0]
    is_L_pizza = [0, 1, 0, 1, 0, 0, 0]
    is_side = [0, 0, 0, 0, 1, 1, 1]
    poteto = [0, 0, 0, 0, 1, 0, 0]

    #######################################################
    # 定式化
    #######################################################
    problem = pulp.LpProblem()
    
    # 変数
    order = [pulp.LpVariable('Order{0}'.format(j), cat=pulp.LpInteger, lowBound=0) for j in range(n)]
    order_price = pulp.lpDot(order, menu_price)
    coupon_use = [pulp.LpVariable('CouponUse{0}'.format(i), cat=pulp.LpInteger, lowBound=0) for i in range(m)]
    piza2set_use = pulp.LpVariable('Piza2SetUse', cat=pulp.LpInteger, lowBound=0)
    pizaL2set_use = pulp.LpVariable('PizaL2SetUse', cat=pulp.LpInteger, lowBound=0)
    piza2set_item = [0, 0, 0, 0, piza2set_use, 0, 0]
    pizaL2set_item = [0, 0, 0, 0, 
                      pulp.LpVariable('PizaL2Set1', cat=pulp.LpInteger, lowBound=0),
                      pulp.LpVariable('PizaL2Set2', cat=pulp.LpInteger, lowBound=0),
                      pulp.LpVariable('PizaL2Set3', cat=pulp.LpInteger, lowBound=0)
                     ]
    payout = order_price - pulp.lpDot(coupon_use, coupon_price)

    # 最小化問題
    problem += payout - pulp.lpSum(coupon_use)
    
    # 制約条件
    problem += pulp.lpDot(coupon_use, coupon_price) <= order_price
    problem += pulp.lpSum(coupon_use) <= (order_price - 1000) 
    for i in range(m):
        problem += coupon_posession[i] - coupon_use[i] >= 0
    problem += coupon_use[0] <= 2
    problem += coupon_use[1] <= 2
    problem += coupon_use[2] <= 3
    problem += coupon_use[3] <= 1
    problem += coupon_use[3] <= np.dot(order, is_pizza)
    problem += piza2set_use*2 <= pulp.lpDot(order, is_pizza)
    problem += pizaL2set_use*2 <= pulp.lpDot(order, is_L_pizza)
    problem += pulp.lpSum(pizaL2set_item) == pizaL2set_use
    problem += max(pulp.lpSum(coupon_use), 1) + max(piza2set_use + pizaL2set_use, 1) <= 1
    for j in range(n):
        problem += order[j] + piza2set_item[j] + pizaL2set_item[j] == requirements[j]

    status = problem.solve()
    print(pulp.LpStatus[status])
    #print(problem)
    print("Order {0}".format([o.value() for o in order]))
    print("Order Price {0}".format(order_price.value()))
    print("Piza2 Set {0}".format(piza2set_use.value()))
    print("PizaL2 Set {0}".format(pizaL2set_use.value()))
    print("Coupon Use {0}".format(sum([coupon_use[i].value() for i in range(m)])))
    return [coupon_use[i].value() for i in range(m)]

In [128]:
# テストケース1
assert solve_level_3(requirements=[1, 0, 1, 0, 1, 0, 0], coupon_posession=[1, 0, 0, 0]) == [1, 0, 0, 0]

Optimal
Order [1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0]
Order Price 2700.0
Piza2 Set 0.0
PizaL2 Set 0.0
Coupon Use 1.0


In [129]:
# テストケース2
assert solve_level_3(requirements=[0, 1, 0, 1, 0, 0, 1], coupon_posession=[1, 0, 0, 0])  == [0, 0, 0, 0]

Optimal
Order [0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]
Order Price 3300.0
Piza2 Set 0.0
PizaL2 Set 1.0
Coupon Use 0.0
